Demostration of `fmslib` in combination with discrete event simulation in `SimPy`

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os

In [2]:
sys.path.append('..')

In [3]:
import simpy
import random
from fsmlib.fsm import FSM, get_random_next_event, parse_transition_func

In [4]:
def get_process(env, name, duration_func):
    
    def p_gen(env):
        while True:
            
            t = duration_func()
            print('[{0}] {1}_start'.format(env.now, name))
            yield env.timeout(t)
            print('[{0}] {1}_stop'.format(env.now, name))
            
    p = env.process(p_gen(env))
    return p
    

In [23]:
env1 = simpy.Environment()

p1 = get_process(env1, 'p1', lambda: 5)
p2 = get_process(env1, 'p2', lambda: 5 + random.random())

env1.run(until=20)


[0] p1_start
[0] p2_start
[5] p1_stop
[5] p1_start
[5.211627717195494] p2_stop
[5.211627717195494] p2_start
[10] p1_stop
[10] p1_start
[10.865863888069887] p2_stop
[10.865863888069887] p2_start
[15] p1_stop
[15] p1_start
[16.727235559820826] p2_stop
[16.727235559820826] p2_start


In [14]:
class SimpleProcess:
    '''
    An independent process with the defined duration function
    '''

    def __init__(self, name, env, duration_func, fsm):
        self.name = name
        self.env = env
        self.duration_func = duration_func
        self.fsm = fsm
        
        self.stats = {'total_time': 0}
        
        self.action = env.process(self.p_gen())
    
    def p_gen(self):
        while True:
            
            next_event = get_random_next_event(self.fsm)
            next_state = self.fsm.transition_func[(self.fsm.current_state, next_event)]
            
            t = self.duration_func[next_state]()
            self.stats['total_time'] += t
            
            self.fsm.advance(next_event)
            
            print('now={0:.4f}\t {1}:{2}!->{3} (will take {4})'.format(self.env.now, self.name, next_event, self.fsm.current_state, t))
            yield self.env.timeout(t)
            
    def print_event(self, event_name):
        print('[{0}] {1}_{2}'.format(self.env.now, self.name, event_name))
        
class ResourceUser(SimpleProcess):
    def process_cycle(self):
        print('Hello')
        super(ResourceUser, self).process_cycle()

In [15]:
env2 = simpy.Environment()

simple_fsm = FSM('idle', {('idle', 'start'): 'running', ('running', 'stop'): 'idle'})

p1 = SimpleProcess('p1', env2, {'idle': lambda: 1, 'running': lambda: 5}, simple_fsm)
p2 = SimpleProcess('p2', env2, {'idle': lambda: 1, 'running': lambda: 5 + random.random()}, simple_fsm)

env2.run(until=50)

now=0.0000	 p1:start!->running (will take 5)
now=0.0000	 p2:stop!->idle (will take 1)
now=1.0000	 p2:start!->running (will take 5.544253987931847)
now=5.0000	 p1:stop!->idle (will take 1)
now=6.0000	 p1:start!->running (will take 5)
now=6.5443	 p2:stop!->idle (will take 1)
now=7.5443	 p2:start!->running (will take 5.99492565456087)
now=11.0000	 p1:stop!->idle (will take 1)
now=12.0000	 p1:start!->running (will take 5)
now=13.5392	 p2:stop!->idle (will take 1)
now=14.5392	 p2:start!->running (will take 5.1114202555462676)
now=17.0000	 p1:stop!->idle (will take 1)
now=18.0000	 p1:start!->running (will take 5)
now=19.6506	 p2:stop!->idle (will take 1)
now=20.6506	 p2:start!->running (will take 5.088153757087197)
now=23.0000	 p1:stop!->idle (will take 1)
now=24.0000	 p1:start!->running (will take 5)
now=25.7388	 p2:stop!->idle (will take 1)
now=26.7388	 p2:start!->running (will take 5.419299071779552)
now=29.0000	 p1:stop!->idle (will take 1)
now=30.0000	 p1:start!->running (will take 5)
n

In [16]:
p1.stats, p2.stats

({'total_time': 53}, {'total_time': 51.71776720456695})

In [17]:
env = simpy.Environment()
rsc = simpy.Resource(env, capacity=1)

In [18]:
d1 = {'a': 1, 'b': 2, 'd': 4}
d2 = {'a': 1, 'c': 12, 'e': 31}


common = []
diff1 = []
diff2 = []
for el in d1:
    if el in d2:
        common.append(el)
    else:
        diff1.append(el)

for el in d2:
    if el not in d1:
        diff2.append(el)
        
print(common)
print(diff1)
print(diff2)


['a']
['b', 'd']
['c', 'e']


In [ ]:
lst = [0, -10, 5, 6, -4]
def f(i):
    return i+2
def g(i):
    return i>5


for el in lst:
    incr = f(el)
    print(incr, g(incr))

In [19]:
lst = [1, 3, ['hello', 4, [[13.2, 45, 'world']], 'boo-ya!', 31], 12]
lst

[1, 3, ['hello', 4, [[13.2, 45, 'world']], 'boo-ya!', 31], 12]

In [20]:
def flat_list(lst):

    new_list = []
    for el in lst:
        if not type(el) is list:
            new_list.append(el)
        else:
            unwrapped = flat_list(el)
            new_list = new_list + unwrapped
    
    return new_list
            
flat_list(lst)

[1, 3, 'hello', 4, 13.2, 45, 'world', 'boo-ya!', 31, 12]